In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [4]:
pd.__version__

'1.4.2'

In [5]:
!pip install pyarrow

In [6]:
# df2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")


In [7]:
# df2

In [8]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

In [9]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [10]:
df1["duration"] = (df1["tpep_dropoff_datetime"] - df1["tpep_pickup_datetime"])/pd.to_timedelta(1, unit="m")

In [ ]:
"Q1: "+str(df.co)

In [11]:
print("Q2: {deviation:.2f}%".format(deviation = df1["duration"].std()))

Q2: 42.59%


In [12]:
df1.duration.describe(percentiles=[0.01, 0.02, 0.95, 0.98, 0.99, 0.995])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
1%       7.833333e-01
2%       2.050000e+00
50%      1.151667e+01
95%      3.646667e+01
98%      4.873333e+01
99%      5.725000e+01
99.5%    6.551667e+01
max      1.002918e+04
Name: duration, dtype: float64

In [13]:
df = df1[ (df1["duration"]>=1) & (df1["duration"] <= 60)]

In [17]:
print("Q3: {fraction:.2f}%".format(fraction = (len(df)/len(df1))*100))

Q3: 98.12%


In [18]:
# sns.distplot(df.duration)

In [19]:
# df1.duration.describe(percentiles=[0.95, 0.98, 0.99, 0.999])

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

In [20]:
# df.head()

In [21]:
df = df.copy()

In [22]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [23]:
df[categorical] = df[categorical].astype(str)

In [24]:
# df.dtypes

In [25]:
# df[categorical + numerical].iloc[:10].to_dict(orient='records')

In [26]:
train_dict = df[categorical].to_dict(orient='records')

In [27]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

In [28]:
print("Q4: "+str(len(dv.feature_names_)))

Q4: 515


In [29]:
target = 'duration'
y_train = df[target].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [30]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [31]:
y_predict = lr.predict(X_train)

In [32]:
#sns.distplot(y_predict, label='prediction')
#sns.distplot(y_train, label='train')
#plt.legend

In [33]:
rmse = mean_squared_error(y_train, y_predict, squared=False)

In [34]:
"Q5: {rmse:.3f}".format(rmse = rmse)

'Q5: 7.649'

In [35]:
# X_train

In [36]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])/pd.to_timedelta(1, unit="m")
    df = df[ (df["duration"]>=1) & (df["duration"] <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [37]:
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
val_dicts = df_val[categorical].to_dict(orient='records')

In [38]:
X_val = dv.transform(val_dicts)

In [39]:
y_predict = lr.predict(X_val)
y_train = df_val[target].values

In [40]:
rmse = mean_squared_error(y_train, y_predict, squared=False)

In [42]:
"Q6: {rmse:.3f}".format(rmse = rmse)

'Q6: 7.812'